<a href="https://colab.research.google.com/github/abhiWriteCode/dl_models/blob/master/vgg19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# VGG19

[paper](https://arxiv.org/abs/1409.1556)

In [1]:
#@title Install PyTorch
!pip install pytorch torchvision

    100% |████████████████████████████████| 2.0MB 11.8MB/s 
  Failed building wheel for pytorch
  Running setup.py clean for pytorch
Failed to build pytorch
imgaug 0.2.8 has requirement numpy>=1.15.0, but you'll have numpy 1.14.6 which is incompatible.
fastai 1.0.46 has requirement numpy>=1.15, but you'll have numpy 1.14.6 which is incompatible.
albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.8 which is incompatible.
  Running setup.py install for pytorch ... error
Command "/usr/bin/python3 -u -c "import setuptools, tokenize;__file__='/tmp/pip-install-aplalin7/pytorch/setup.py';f=getattr(tokenize, 'open', open)(__file__);code=f.read().replace('\r\n', '\n');f.close();exec(compile(code, __file__, 'exec'))" install --record /tmp/pip-record-u0zot99x/install-record.txt --single-version-externally-managed --compile" failed with error code 1 in /tmp/pip-install-aplalin7/pytorch/


## Import

In [0]:
import numpy as np

import torch
from torch import nn
from torch.utils import model_zoo

## Network

In [0]:
class VGG19(nn.Module):
    
    def __init__(self, num_classes=1000, batch_norm=False):
        super().__init__()
        self.conv = VGG19.__make_layers(batch_norm)
        self.avgpool = nn.AdaptiveAvgPool2d((7, 7))
        self.fc = nn.Sequential(
            nn.Linear(512*7*7, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, num_classes),
        )
        self.__initialize_weights()
        
    def forward(self, x):
        x = self.conv(x)
        x = self.avgpool(x)
        x = x.view(x.shape(0), -1)
        x = self.fc(x)
        
        return x
                
    def __initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)
                
    
    def __make_layers(batch_norm):
        cfg = [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 
               'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M']
        layers = []
        in_channels = 3
        for l in cfg:
            if l == 'M':
                layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
            else:
                conv2d = nn.Conv2d(in_channels, l, kernel_size=3, padding=1)
                if batch_norm:
                    layers += [conv2d, nn.BatchNorm2d(l), nn.ReLU(inplace=True)]
                else:
                    layers += [conv2d, nn.ReLU(inplace=True)]
                in_channels = l
                
        return nn.Sequential(*layers)


In [0]:
model = VGG19()
model_bn = VGG19(batch_norm=True)

## Load pre-trained model's weight

In [14]:
#########################################################################
model_path = 'https://download.pytorch.org/models/vgg19-dcbb9e9d.pth'

model.load_state_dict(
    model_zoo.load_url(model_path),
    strict=False
)

#########################################################################

model_path_bn = 'https://download.pytorch.org/models/vgg19_bn-c79401a0.pth'

model_bn.load_state_dict(
    model_zoo.load_url(model_path_bn),
    strict=False
)
#########################################################################

Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.torch/models/vgg19-dcbb9e9d.pth
574673361it [00:19, 28804927.26it/s]
Downloading: "https://download.pytorch.org/models/vgg19_bn-c79401a0.pth" to /root/.torch/models/vgg19_bn-c79401a0.pth
574769405it [00:07, 72760927.48it/s]
